# **Libs**

In [65]:
# Utils
from src.utils import *

In [80]:
import nltk
import torch
from torch.utils.data import Dataset, DataLoader
from nltk.corpus import stopwords
from collections import Counter

# Загрузка стоп-слов для русского языка
nltk.download('stopwords')
stop_words = set(stopwords.words('russian'))

[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\Sefixnep\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


# **Code**

## Зафиксируем seed

In [68]:
set_all_seeds()

In [69]:
df = pd.read_csv("../data/train.csv")

In [70]:
texts = df['comment_text'].tolist()
labels = df['toxic'].tolist()

In [86]:
import torch
from torch.utils.data import Dataset
from transformers import BertTokenizer

class TextDataset(Dataset):
    def __init__(self, texts, labels, tokenizer, max_len):
        self.texts = texts          # Список исходных текстов
        self.labels = labels        # Список меток
        self.tokenizer = tokenizer  # Токенизатор из библиотеки transformers
        self.max_len = max_len      # Максимальная длина последовательности

    def __len__(self):
        return len(self.texts)

    def __getitem__(self, idx):
        # Получаем текст и метку по индексу
        text = self.texts[idx]
        label = self.labels[idx]

        # Токенизация и кодирование текста
        encoding = self.tokenizer.encode_plus(
            text,
            add_special_tokens=True,        # Добавляем специальные токены [CLS] и [SEP]
            max_length=self.max_len,
            padding='max_length',           # Дополнение последовательности до max_len
            truncation=True,                # Обрезка последовательностей длиннее max_len
            return_attention_mask=True,     # Возвращаем маску внимания
            return_tensors='pt',            # Возвращаем тензоры PyTorch
        )

        # Возвращаем словарь с тензорами
        return {
            'input_ids': encoding['input_ids'].squeeze(0),          # Убираем дополнительную размерность
            'attention_mask': encoding['attention_mask'].squeeze(0),
            'labels': torch.tensor(label, dtype=torch.long)
        }


In [ ]:
import torch
import torch.nn as nn
from transformers import BertModel

class TextClassifier(nn.Module):
    def __init__(self, n_classes):
        super(TextClassifier, self).__init__()
        self.bert = BertModel.from_pretrained('bert-base-uncased')  # Можно заменить на другую модель BERT
        self.dropout = nn.Dropout(p=0.3)
        self.classifier = nn.Linear(self.bert.config.hidden_size, n_classes)
    
    def forward(self, input_ids, attention_mask):
        # Получаем выходы BERT
        outputs = self.bert(input_ids=input_ids, attention_mask=attention_mask)
        # Используем пуллированный выход [CLS] токена
        pooled_output = outputs.pooler_output
        # Применяем Dropout
        pooled_output = self.dropout(pooled_output)
        # Проходим через классификатор
        logits = self.classifier(pooled_output)
        return logits


In [88]:
Tokenizer = tokenizer.from_pretrained('bert-base-uncased')

BertTokenizer(name_or_path='bert-base-uncased', vocab_size=30522, model_max_length=512, is_fast=False, padding_side='right', truncation_side='right', special_tokens={'unk_token': '[UNK]', 'sep_token': '[SEP]', 'pad_token': '[PAD]', 'cls_token': '[CLS]', 'mask_token': '[MASK]'}, clean_up_tokenization_spaces=True),  added_tokens_decoder={
	0: AddedToken("[PAD]", rstrip=False, lstrip=False, single_word=False, normalized=False, special=True),
	100: AddedToken("[UNK]", rstrip=False, lstrip=False, single_word=False, normalized=False, special=True),
	101: AddedToken("[CLS]", rstrip=False, lstrip=False, single_word=False, normalized=False, special=True),
	102: AddedToken("[SEP]", rstrip=False, lstrip=False, single_word=False, normalized=False, special=True),
	103: AddedToken("[MASK]", rstrip=False, lstrip=False, single_word=False, normalized=False, special=True),
}